# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

scooter rider injured in hit and run 

https://www.denver7.com/news/local-news/denver-woman-recovering-from-serious-injuries-after-hit-and-run-while-riding-scooter

incident_id = 2023182255

In [3]:
# University Blvd and E 3rd Ave (2022,12,14) incident_id = 2022639830
# lat_lon = [39.720934, -104.959436]

# Speer Blvd and W 11th Ave
# lat_lon = [39.73378086237446, -104.99382870794332]

# Sheridan and W 38th Ave
# lat_lon = [39.76932995840248, -105.05322879106377]

# 18th St and Wazee - 2023-01-07
# lat_lon = [39.753095972145644, -104.99765515593928]

# 17th Ave and Humboldt
# lat_lon = [39.74325582993406, -104.96969463423684]

# Evans and Holly
# lat_lon = [39.678456222120396, -104.92219658639499]

# Perry and 32nd
# lat_lon = [39.76207714280117, -105.03931855940714]

# W 23rd Ave and Julian
# lat_lon = [39.751187102730654, -105.03155988650708]

# 35th St and Welton
# lat_lon = [39.76762137425839, -104.97535848218475]

# 1776 Race St
# lat_lon = [39.744484814811436, -104.96299454652576]

# 730 S Canosa Ct
# lat_lon = [39.70332702509137, -105.01894998129806]

# Sherman St house
# lat_lon = [39.721383, -104.984736]

# Broadway and Colfax - scooter rider injured in hit and run
# lat_lon = [39.74012825562107, -104.98743057571703]

# lat_lon = [39.68921698548907, -105.00035211657175] # West Florida Ave and South Platte River Drive West, Denver, CO

# lat_lon = cda.geocode_location('W 14th Ave & Federal Blvd, Denver, CO')
# lat_lon = cda.geocode_location('W 32nd Ave & Speer Blvd, Denver, CO')
# lat_lon = cda.geocode_location('Columbine and E 14th Ave, Denver, CO')
# lat_lon = cda.geocode_location('1360 S. Bryant St., Denver, CO')

geocoded_location = 'Speer and Downing, Denver, CO'

In [4]:
lat_lon = cda.geocode_location(geocoded_location)

# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.1

query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

nearby['reported_date_local'] = nearby['reported_date_local'].dt.tz_localize('America/Denver')

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,29,0,0,0,0
2014,43,0,1,0,1
2015,56,1,0,1,2
2016,37,1,0,1,2
2017,38,0,0,1,1
2018,37,0,1,0,2
2019,31,0,0,0,0
2020,21,0,0,0,0
2021,22,0,0,0,0


In [6]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,376,2,5,3,10


In [7]:
display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
    , 'distance_feet'
    , 'incident_id'
]
# nearby.to_clipboard()
# nearby[display_cols]

In [8]:
nearby[display_cols] #.tail(5)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle_ind,pedestrian_ind,sbi,fatality,distance_feet,incident_id
0,2013-01-10 08:25:00-07:00,TRAF - ACCIDENT,N CORONA ST / E SPEER BLVD,0.0,0.0,False,False,227.032835,201314946
1,2013-01-10 14:38:00-07:00,TRAF - ACCIDENT,N DOWNING ST / E SPEER BLVD,0.0,0.0,False,False,115.585462,201315411
2,2013-02-20 23:28:00-07:00,TRAF - ACCIDENT - HIT & RUN,E SPEER BLVD / N DOWNING ST,0.0,0.0,False,False,228.720094,201384507
3,2013-03-11 09:44:00-06:00,TRAF - ACCIDENT - HIT & RUN,100 BLOCK N DOWNING ST,0.0,0.0,False,False,270.394473,2013113627
4,2013-03-21 12:14:00-06:00,TRAF - ACCIDENT,N DOWNING ST / E 1ST AVE,0.0,0.0,False,False,115.585462,2013129509
...,...,...,...,...,...,...,...,...,...
371,2023-11-08 11:11:00-07:00,TRAF - ACCIDENT,N DOWNING ST / E SPEER BLVD,0.0,0.0,False,False,115.585462,2023603083
372,2023-11-08 22:33:00-07:00,TRAF - ACCIDENT - SBI,E 1ST AVE / N DOWNING ST,0.0,0.0,True,False,192.480126,2023604136
373,2023-11-20 09:00:00-07:00,TRAF - ACCIDENT,E SPEER BLVD / N DOWNING ST,0.0,0.0,False,False,228.720094,2023625104
374,2023-12-16 03:02:00-07:00,TRAF - ACCIDENT,E SPEER BLVD / N CORONA ST,0.0,0.0,False,False,228.720094,2023670222


In [9]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [12]:
# Frequency of crashes
num_years = 3

most_recent_timestamp = cda.most_recent_crash_timestamp()
start_timestamp = most_recent_timestamp - pd.DateOffset(years=num_years)
days_in_time_range = (most_recent_timestamp - start_timestamp).days

nearby_in_time_range = nearby[nearby.reported_date_local > start_timestamp].copy()

days_between_crashes = days_in_time_range / len(nearby_in_time_range)

print(
    f'The location "{geocoded_location}" has had {len(nearby_in_time_range)} traffic crashes'
    , f'within a {radius_miles} mile radius'
    , f'over the last {num_years} years, an average of {days_between_crashes:.1f} days between crashes.'
    , f'In that time, there were {nearby_in_time_range.fatality.sum()} deadly crashes'
    , f'and {nearby_in_time_range.sbi.sum()} crashes causing a serious bodily injury.'
    , f'There were {(nearby_in_time_range.pedestrian_ind > 0).sum()} crashes involving a pedestrian'
    , f'and {(nearby_in_time_range.bicycle_ind > 0).sum()} crashes involving a bicyclist.'
)

The location "Speer and Downing, Denver, CO" has had 84 traffic crashes within a 0.1 mile radius over the last 3 years, an average of 13.0 days between crashes. In that time, there were 0 deadly crashes and 3 crashes causing a serious bodily injury. There were 0 crashes involving a pedestrian and 2 crashes involving a bicyclist.


In [11]:
cda.conn.dispose()

,incident_id,top_traffic_accident_offense,reported_date,incident_address_corrected,at_freeway,geo_lon,geo_lat,neighborhood_id,bicycle_ind,pedestrian_ind,...,crash_date,crash_date_str,crash_time_str,crash_year,crash_day_of_year,geom,geom_denver,reported_date_local,distance_feet,all_time
0,201314946,TRAF - ACCIDENT,2013-01-10 15:25:00+00:00,N CORONA ST / E SPEER BLVD,False,-104.974055,39.718886,Speer,0.0,0.0,...,2013-01-10,2013-01-10 Thu,"Thu Jan 10, 8:25 AM",2013,10,0101000020E61000002706F5EB563E5AC06F055E7704DC...,0101000020AE0D000015FFCC7F57044841D98759007ABE...,2013-01-10 08:25:00-07:00,227.032835,all time
1,201315411,TRAF - ACCIDENT,2013-01-10 21:38:00+00:00,N DOWNING ST / E SPEER BLVD,False,-104.972888,39.718620,Country Club,0.0,0.0,...,2013-01-10,2013-01-10 Thu,"Thu Jan 10, 2:38 PM",2013,10,0101000020E61000005748D0CA433E5AC044F2BEBFFBDB...,0101000020AE0D0000FDC21200FC0448414C2A38001BBE...,2013-01-10 14:38:00-07:00,115.585462,all time
2,201384507,TRAF - ACCIDENT - HIT & RUN,2013-02-21 06:28:00+00:00,E SPEER BLVD / N DOWNING ST,False,-104.973914,39.718334,Speer,0.0,0.0,...,2013-02-20,2013-02-20 Wed,"Wed Feb 20, 11:28 PM",2013,51,0101000020E6100000C071009A543E5AC09C272A5DF2DB...,0101000020AE0D0000CA32DEFF6B04484153D60B00B1BD...,2013-02-20 23:28:00-07:00,228.720094,all time
3,2013113627,TRAF - ACCIDENT - HIT & RUN,2013-03-11 15:44:00+00:00,100 BLOCK N DOWNING ST,False,-104.972790,39.719364,Country Club,0.0,0.0,...,2013-03-11,2013-03-11 Mon,"Mon Mar 11, 9:44 AM",2013,70,0101000020E6100000AFD8BA2F423E5AC09176ED1C14DC...,0101000020AE0D00005A72E9FF080548418420C9FF29BF...,2013-03-11 09:44:00-06:00,270.394473,all time
4,2013129509,TRAF - ACCIDENT,2013-03-21 18:14:00+00:00,N DOWNING ST / E 1ST AVE,False,-104.972888,39.718620,Country Club,0.0,0.0,...,2013-03-21,2013-03-21 Thu,"Thu Mar 21, 12:14 PM",2013,80,0101000020E61000005748D0CA433E5AC044F2BEBFFBDB...,0101000020AE0D0000FDC21200FC0448414C2A38001BBE...,2013-03-21 12:14:00-06:00,115.585462,all time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2023603083,TRAF - ACCIDENT,2023-11-08 18:11:00+00:00,N DOWNING ST / E SPEER BLVD,False,-104.972888,39.718620,Country Club,0.0,0.0,...,2023-11-08,2023-11-08 Wed,"Wed Nov 8, 11:11 AM",2023,312,0101000020E61000005748D0CA433E5AC044F2BEBFFBDB...,0101000020AE0D0000FDC21200FC0448414C2A38001BBE...,2023-11-08 11:11:00-07:00,115.585462,all time
372,2023604136,TRAF - ACCIDENT - SBI,2023-11-09 05:33:00+00:00,E 1ST AVE / N DOWNING ST,False,-104.973232,39.718196,Speer,0.0,0.0,...,2023-11-08,2023-11-08 Wed,"Wed Nov 8, 10:33 PM",2023,312,0101000020E61000002B64956F493E5AC04BD424DBEDDB...,0101000020AE0D00007EFFCBFFCB0448415F5FF5FF7FBD...,2023-11-08 22:33:00-07:00,192.480126,all time
373,2023625104,TRAF - ACCIDENT,2023-11-20 16:00:00+00:00,E SPEER BLVD / N DOWNING ST,False,-104.973914,39.718334,Speer,0.0,0.0,...,2023-11-20,2023-11-20 Mon,"Mon Nov 20, 9:00 AM",2023,324,0101000020E6100000C071009A543E5AC09C272A5DF2DB...,0101000020AE0D0000CA32DEFF6B04484153D60B00B1BD...,2023-11-20 09:00:00-07:00,228.720094,all time
374,2023670222,TRAF - ACCIDENT,2023-12-16 10:02:00+00:00,E SPEER BLVD / N CORONA ST,False,-104.973914,39.718334,Speer,0.0,0.0,...,2023-12-16,2023-12-16 Sat,"Sat Dec 16, 3:02 AM",2023,350,0101000020E6100000C071009A543E5AC09C272A5DF2DB...,0101000020AE0D0000CA32DEFF6B04484153D60B00B1BD...,2023-12-16 03:02:00-07:00,228.720094,all time
